In [ ]:
%pip install -r requirements.txt

In [66]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

Create the Azure Index Search

In [ ]:
from azure.identity import DefaultAzureCredential

from azure.identity import get_bearer_token_provider
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchIndex
)
import os

credential = DefaultAzureCredential()

# Create a search index  
index_name = "recipe_book_idx"
index_client = SearchIndexClient(endpoint=os.getenv('AZURE_AI_SEARCH_ENDPOINT'), credential=credential)  
fields = [    
    SearchField(name="id", type=SearchFieldDataType.String,key=True),           
    SearchField(name="country", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=True, filterable=False),  
    SearchField(name="city", type=SearchFieldDataType.String, searchable=True,sortable=False, facetable=False, filterable=False),
    SearchField(name="recipe_name", type=SearchFieldDataType.String,searchable=True,sortable=False, facetable=False, filterable=False),
    SearchField(name="description", type=SearchFieldDataType.String,searchable=True,sortable=False, facetable=False, filterable=False),    
    SearchField(name="vector_description", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=1536, vector_search_profile_name="vector-profile-1",searchable=True,sortable=False, facetable=False, filterable=False),
    SearchField(name="ingredients", type=SearchFieldDataType.Collection(SearchFieldDataType.String),searchable=True,sortable=False, facetable=False, filterable=False),    
    SearchField(name="instructions", type=SearchFieldDataType.String,searchable=True,sortable=False, facetable=False, filterable=False)    
]  

# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="vector-profile-1",  
            algorithm_configuration_name="myHnsw"
        )
    ]
)  

# Create the search index
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")

This code is not needed to run just to show how we create the embedding, if you add more rows run this method

In [ ]:
import json
import uuid
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI

endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
model_name = "text-embedding-3-small"
deployment = "text-embedding-3-small"

token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
api_version = "2024-10-21"

client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = api_version,
  azure_ad_token_provider=token_provider,
)

# Load recipes.json
with open("./recipes/recipes.json", "r") as file:
    recipes = json.load(file)

for recipe in recipes:
    description = recipe.get("description", "")
    if description:
        response = client.embeddings.create(
            input=[description],
            model=deployment
        )
        recipe["id"] = str(uuid.uuid4())
        recipe["vector_description"] = response.data[0].embedding 

with open("updated_recipes.json", "w") as file:
    json.dump(recipes, file, indent=4)          

print("Embeddings generated and saved to updated_recipes.json")     

Uploads all document in Azure AI Search, you need to have the RBAC **Search Index Data Contributor** to have this to work.

In [63]:
# Upload documents in index
from azure.search.documents import SearchClient

output_path = os.path.join('recipes', 'updated_recipes.json')
output_directory = os.path.dirname(output_path)

with open(output_path, 'r') as file:  
    documents = json.load(file)  

credential = DefaultAzureCredential()

# Create a search index  
search_client = SearchClient(endpoint=os.getenv('AZURE_AI_SEARCH_ENDPOINT'), credential=credential, index_name=index_name)

result = search_client.upload_documents(documents)  